Energy system plot

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create directed graph
G = nx.DiGraph()

# Add nodes with attributes
nodes = [
    ("Power Plant", {"type": "Generation", "voltage": "High"}),
    ("Primary Substation", {"type": "Transformation", "voltage": "High/Medium"}),
    ("Industrial Consumer", {"type": "Load", "voltage": "Medium"}),
    ("Transmission Station", {"type": "Transmission", "voltage": "High"}),
    ("Distribution Station", {"type": "Distribution", "voltage": "Medium"}),
    ("Residential Area", {"type": "Load", "voltage": "Low"})
]

G.add_nodes_from(nodes)

# Add edges with attributes
edges = [
    ("Power Plant", "Transmission Station", {"label": "High-Voltage Transmission Line", "length": 50}),
    ("Transmission Station", "Primary Substation", {"label": "High-Voltage Bus", "length": 10}),
    ("Primary Substation", "Industrial Consumer", {"label": "Medium-Voltage Feed", "length": 5}),
    ("Primary Substation", "Distribution Station", {"label": "Medium-Voltage Distribution", "length": 8}),
    ("Distribution Station", "Residential Area", {"label": "Low-Voltage Lines", "length": 3})
]

G.add_edges_from(edges)

# Set positions for visualization
pos = {
    "Power Plant": (0, 2),
    "Transmission Station": (4, 3),
    "Primary Substation": (8, 2),
    "Industrial Consumer": (9, 5),
    "Distribution Station": (4, 0),
    "Residential Area": (8, -1)
}

# Create figure
plt.figure(figsize=(12, 8))
ax = plt.gca() # Get current axes

# Draw nodes with styling
node_colors = {
    "Generation": "green",
    "Transformation": "orange",
    "Transmission": "yellow",
    "Distribution": "blue",
    "Load": "red"
}

node_colors_list = [node_colors.get(G.nodes[n]['type'], 'grey') for n in G.nodes] # Use .get for safety
nx.draw_networkx_nodes(G, pos, node_size=3500, node_color=node_colors_list, alpha=0.9, edgecolors='black') # Slightly larger, add edgecolors

# Draw edges with styling and arrows
edge_styles = {
    "High-Voltage Transmission Line": {"color": "red", "width": 2, "style": "solid"},
    "High-Voltage Bus": {"color": "darkred", "width": 2, "style": "dashed"},
    "Medium-Voltage Feed": {"color": "blue", "width": 1.5, "style": "solid"},
    "Medium-Voltage Distribution": {"color": "blue", "width": 1.5, "style": "dashed"},
    "Low-Voltage Lines": {"color": "gray", "width": 1, "style": "solid"}
}

for edge in G.edges(data=True):
    style = edge_styles.get(edge[2]['label'], {'color': 'black', 'width': 1, 'style': 'solid'}) # Default style
    nx.draw_networkx_edges(G, pos, edgelist=[(edge[0], edge[1])], 
                          edge_color=style['color'], 
                          width=style['width'], 
                          style=style['style'], 
                          arrows=True, arrowstyle='-|>', arrowsize=20) # Add arrows

# Draw labels
nx.draw_networkx_labels(G, pos, font_size=10, font_weight="bold")
edge_labels = nx.get_edge_attributes(G, "label")
# Adjust edge label position slightly to avoid overlap with lines
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=9, label_pos=0.6, verticalalignment='bottom')

# Add legend
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', label='Generation', 
              markerfacecolor='green', markersize=15),
    plt.Line2D([0], [0], marker='o', color='w', label='Transformation', 
              markerfacecolor='orange', markersize=15),
    plt.Line2D([0], [0], marker='o', color='w', label='Transmission', 
              markerfacecolor='yellow', markersize=15),
    plt.Line2D([0], [0], marker='o', color='w', label='Distribution', 
              markerfacecolor='blue', markersize=15),
    plt.Line2D([0], [0], marker='o', color='w', label='Load', 
              markerfacecolor='red', markersize=15)
]

# Position legend outside the main plot area
ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0.)

plt.title("Generation → Transmission → Distribution", fontsize=14)
plt.axis('off')
plt.subplots_adjust(right=0.75) # Make space for the legend on the right
# plt.tight_layout() # tight_layout often conflicts with bbox_to_anchor for legends
# plt.savefig("power_system_network.png", dpi=300, bbox_inches='tight') # Use bbox_inches='tight' when saving
plt.show()

Physics of flow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# Set up a simple power system as a graph
G = nx.DiGraph()

# Add nodes (buses) with voltage levels
buses = {
    "Gen": {"voltage": 100, "type": "generator"},
    "Load1": {"voltage": 95, "type": "load"},
    "Load2": {"voltage": 92, "type": "load"},
    "Substation": {"voltage": 98, "type": "substation"},
}
for node, attrs in buses.items():
    G.add_node(node, **attrs)

# Add edges (transmission lines) with resistance and reactance
# Note: In real AC systems, X is often > R for transmission lines.
lines = [
    ("Gen", "Substation", {"resistance": 0.1, "reactance": 0.5, "rating": 15}), # Lower R, higher X, add rating (e.g., MVA)
    ("Substation", "Load1", {"resistance": 0.2, "reactance": 1.0, "rating": 10}),
    ("Substation", "Load2", {"resistance": 0.3, "reactance": 1.5, "rating": 10}),
]
for u, v, attrs in lines:
    G.add_edge(u, v, **attrs)

# Position nodes for visualization
pos = {
    "Gen": (0, 1),
    "Substation": (1, 1),
    "Load1": (2, 0.5),
    "Load2": (2, 1.5),
}

# Draw the graph
plt.figure(figsize=(10, 6))
nx.draw_networkx_nodes(
    G, pos, node_color="lightblue", node_size=2000, edgecolors="black"
)
nx.draw_networkx_labels(G, pos, font_size=12, font_weight="bold")

# Draw edges with parameters
# Full AC power flow calculation (P, Q, V, δ) is complex and depends on load/gen.
# Here, we just label the line parameters (R, X).
edge_labels = {}
for u, v, data in G.edges(data=True):
    label = f"R={data['resistance']} Ω\nX={data['reactance']} Ω"
    # label = f"R={data['resistance']}\nX={data['reactance']}\nRating={data['rating']} MVA" # Example including rating
    edge_labels[(u, v)] = label

nx.draw_networkx_edges(
    G, pos, edge_color="gray", width=2, arrowstyle="->", arrowsize=20
)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)

# Annotate Ohm's Law and Kirchhoff's Laws
plt.text(
    0.5, 0.1,
    "Power Network\n"
    "Lines represented by Impedance Z = R + jX\n"
    "AC Power Flow depends on V magnitudes and angles",
    ha="center", va="center", fontsize=12, bbox=dict(facecolor="white", alpha=0.8)
)

plt.title("Power System Network Representation (AC Parameters)", fontsize=14)
plt.axis("off")
plt.tight_layout()

# Save the figure
# plt.savefig("power_system_flow.png", dpi=300, bbox_inches="tight")
plt.show()

Energy flow

In [ ]:
import plotly.graph_objects as go

# Define energy flows (in arbitrary units, e.g., GWh)
generation = 50
imports = 20
storage_discharge = 10
load = 60
exports = 5
storage_charging = 8
losses = 7

# Validate energy balance (generation + imports + discharge = load + exports + charging + losses)
assert (
    generation + imports + storage_discharge
    == load + exports + storage_charging + losses
), "Energy balance equation is not satisfied!"

# Sankey diagram setup
fig = go.Figure(go.Sankey(
    arrangement="snap",
    node={
        "label": [
            "Generation", "Imports", "Storage (Discharge)",  # Sources
            "Load", "Exports", "Storage (Charge)", "Losses"  # Sinks
            "Supply", "Demand"  # Aggregators
        ],
        "color": [
            "green", "blue", "orange",  # Sources
            "red", "purple", "cyan", "gray"  # Sinks
            "lightgreen", "lightcoral"  # Aggregators
        ],
    },
    link={
        "source": [
            0, 1, 2,  # Sources -> Supply
            7, 7, 7, 7  # Demand -> Sinks
        ],
        "target": [
            7, 7, 7,  # Sources -> Supply
            3, 4, 5, 6  # Demand -> Load, Exports, etc.
        ],
        "value": [
            generation, imports, storage_discharge,  # Flows into Supply
            load, exports, storage_charging, losses  # Flows out of Demand
        ],
        "color": [
            "rgba(0, 128, 0, 0.3)", "rgba(0, 0, 255, 0.3)", "rgba(255, 165, 0, 0.3)",  # Source colors
            "rgba(255, 0, 0, 0.3)", "rgba(128, 0, 128, 0.3)", "rgba(0, 255, 255, 0.3)", "rgba(128, 128, 128, 0.3)"  # Sink colors
        ],
    }
))

# Update layout
fig.update_layout(
    title="Generation + Imports + Storage Discharge = Load + Exports + Storage Charging + Losses<br>",
        #   "<sup></sup>",
    font=dict(size=10),
    height=600
)

# Save as HTML or display
# fig.write_html("energy_balance.html")
fig.show()

Energy Balance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data: demand, wind, solar, backup, curtailment with added variability
np.random.seed(42) # for reproducibility
time = np.linspace(0, 24, 100)

# More realistic demand: base + peak + noise
base_demand = 80
peak_demand = 40 * np.sin(2 * np.pi * (time - 9) / 24) # Afternoon peak shifted
demand_noise = np.random.normal(0, 3, time.shape) # Add random fluctuations
demand = base_demand + np.maximum(0, peak_demand) + demand_noise

# More realistic wind: base profile + gustiness (noise)
wind_profile = 50 * (np.sin(2 * np.pi * (time - 6) / 24) + 0.8) / 1.8 # Base availability
wind_noise = np.random.normal(0, 5, time.shape) # Gustiness
wind = np.maximum(0, wind_profile + wind_noise)

# More realistic solar: profile based on sun angle + cloudiness (noise)
solar_profile = 60 * np.maximum(0, np.cos(2 * np.pi * (time - 12.5) / 24))**1.5 # Steeper rise/fall around noon
solar_noise = np.random.normal(0, 4, time.shape)
cloud_factor = np.maximum(0.2, 1 - 0.8 * np.random.rand(time.shape[0])**2) # Random cloud cover reducing output
solar = np.maximum(0, solar_profile * cloud_factor + solar_noise)

# Recalculate balance
generation = wind + solar
mismatch = demand - generation
backup = np.maximum(mismatch, 0)
curtailment = np.maximum(-mismatch, 0)

# Plotting the balance between demand and supply
plt.figure(figsize=(12, 7))
plt.plot(time, demand, label='Demand', color='black', linewidth=2)
plt.plot(time, wind, label='Wind Generation', color='deepskyblue', alpha=0.8)
plt.plot(time, solar, label='Solar Generation', color='orange', alpha=0.8)
plt.plot(time, generation, label='Total VRE Gen', color='green', linestyle='--', linewidth=1.5)

# Use fill_between for backup and curtailment based on mismatch
plt.fill_between(time, demand, generation, where=demand > generation, color='red', alpha=0.4, label='Energy Deficit (Backup Need)')
plt.fill_between(time, generation, demand, where=generation > demand, color='purple', alpha=0.4, label='Excess Energy (Curtailment/Storage)')

plt.title('Balancing Variable Demand and Renewable Generation (Illustrative)')
plt.xlabel('Time (hours)')
plt.ylabel('Power (MW)')
plt.legend(loc='upper left')
plt.grid(True, linestyle=':', alpha=0.7)
plt.xlim(0, 24)
plt.ylim(bottom=0)
plt.tight_layout()
plt.show()


Demand side management

In [ ]:
import numpy as np

# --- Regenerate Base Data for Consistent Comparison ---
np.random.seed(42) # for reproducibility
time = np.linspace(0, 24, 100)
time_step = time[1] - time[0] # Hours per time step

# Realistic demand: base + peak + noise
base_demand = 80
peak_demand_factor = 45 * np.sin(np.pi * (time - 8) / 12) # Peak centered around 2 PM (hour 14)
demand_noise = np.random.normal(0, 4, time.shape)
demand = base_demand + np.maximum(0, peak_demand_factor) + demand_noise
demand = np.maximum(demand, 20) # Min load

# Realistic wind: base profile + gustiness
wind_profile = 55 * (np.sin(2 * np.pi * (time - 6) / 24) + 0.8) / 1.8
wind_noise = np.random.normal(0, 6, time.shape)
wind = np.maximum(0, wind_profile + wind_noise)

# Realistic solar: profile + cloudiness
solar_profile = 70 * np.maximum(0, np.cos(2 * np.pi * (time - 12.5) / 24))**1.8
solar_noise = np.random.normal(0, 5, time.shape)
cloud_factor = np.maximum(0.3, 1 - 0.7 * np.random.rand(time.shape[0])**2)
solar = np.maximum(0, solar_profile * cloud_factor + solar_noise)

# Total renewable generation
generation = wind + solar

# --- Define DSM Strategies & Parameters ---
peak_threshold = 110 # kW, for shaving/shifting
efficiency_factor = 0.85 # 15% reduction
peak_indices = np.where((time >= 12) & (time <= 18) & (demand > peak_threshold))[0] # Peak ~noon-6pm
target_shift_indices = np.where((time >= 18) & (time <= 22))[0] # Target evening hours 6pm-10pm

# 1. Peak Shaving Profile
load_peak_shaving = np.minimum(demand, peak_threshold)

# 2. Load Shifting Profile (to Evening)
load_shifted_evening = demand.copy()
energy_to_shift = 0
if len(peak_indices) > 0:
    energy_above_threshold = (demand[peak_indices] - peak_threshold) * time_step
    energy_to_shift = np.sum(energy_above_threshold)
    load_shifted_evening[peak_indices] = peak_threshold # Cap the peak

if len(target_shift_indices) > 0 and energy_to_shift > 0:
    target_duration = len(target_shift_indices) * time_step
    power_increase_target = energy_to_shift / target_duration
    load_shifted_evening[target_shift_indices] += power_increase_target

# 3. Efficiency Improvement Profile
load_efficiency = demand * efficiency_factor

# --- Plot 1: Illustrate Load Shifting Concept ---
plt.figure(figsize=(10, 5))
plt.plot(time, demand, label='Original Demand', color='black', linestyle=':')
plt.plot(time, load_shifted_evening, label='Shifted Demand', color='blue')
# Highlight removed peak
plt.fill_between(time, demand, load_shifted_evening, where=(demand > load_shifted_evening) & (demand > peak_threshold), 
                 color='lightcoral', alpha=0.6, label='Energy Shifted FROM Peak')
# Highlight added off-peak
plt.fill_between(time, load_shifted_evening, demand, where=(load_shifted_evening > demand), 
                 color='lightblue', alpha=0.7, label='Energy Shifted TO Off-Peak')
plt.title('DSM: Load Shifting (Peak to Evening)', fontsize=14)
plt.xlabel('Time (hours)')
plt.ylabel('Power (kW)')
plt.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.ylim(bottom=0)
plt.xlim(0, 24)
plt.tight_layout()
plt.show()

# --- Plot 2: Illustrate Efficiency Measures Concept ---
plt.figure(figsize=(10, 5))
plt.plot(time, demand, label='Original Demand', color='black', linestyle=':')
plt.plot(time, load_efficiency, label=f'Efficient Demand ({int((1-efficiency_factor)*100)}% Reduction)', color='green')
plt.fill_between(time, demand, load_efficiency, color='lightgreen', alpha=0.6, label='Energy Saved')
plt.title('DSM: Efficiency Measures', fontsize=14)
plt.xlabel('Time (hours)')
plt.ylabel('Power (kW)')
plt.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.ylim(bottom=0)
plt.xlim(0, 24)
plt.tight_layout()
plt.show()

# --- Plot 3: Illustrate Peak Shaving Concept ---
plt.figure(figsize=(10, 5))
plt.plot(time, demand, label='Original Demand', color='black', linestyle=':')
plt.plot(time, load_peak_shaving, label=f'Shaved Demand (Cap @ {peak_threshold}kW)', color='red')
plt.fill_between(time, demand, load_peak_shaving, where=(demand > peak_threshold),
                 color='lightcoral', alpha=0.6, label='Peak Energy Shaved')
plt.axhline(peak_threshold, color='red', linestyle='--', linewidth=0.8, label='Shaving Threshold')
plt.title('DSM: Peak Shaving', fontsize=14)
plt.xlabel('Time (hours)')
plt.ylabel('Power (kW)')
plt.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.ylim(bottom=0)
plt.xlim(0, 24)
plt.tight_layout()
plt.show()

# --- Create 2x2 Subplots (Existing Combined Impact Plot) ---
fig, axs = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
fig.suptitle('Impact of DSM Strategies on Meeting Demand with Renewables', fontsize=16)

# Helper function to plot a scenario
def plot_scenario(ax, title, demand_profile, gen_profile, time_vector):
    ax.plot(time_vector, gen_profile, label='Renewable Gen (Solar+Wind)', color='green', alpha=0.7)
    ax.plot(time_vector, demand_profile, label='Demand Profile', color='black', linestyle='--')
    
    # Fill deficit (Demand > Generation)
    ax.fill_between(time_vector, demand_profile, gen_profile, where=demand_profile > gen_profile, 
                     color='red', alpha=0.4, label='Deficit (Need Backup/Storage)')
    # Fill surplus (Generation > Demand)
    ax.fill_between(time_vector, demand_profile, gen_profile, where=gen_profile >= demand_profile, 
                     color='lightgreen', alpha=0.4, label='Surplus (Potential Curtailment/Storage)')
    
    ax.set_title(title)
    ax.grid(True, linestyle=':', alpha=0.6)
    ax.set_ylim(bottom=0)
    ax.set_xlim(0, 24)

# Plot each scenario
plot_scenario(axs[0, 0], 'Baseline (No DSM)', demand, generation, time)
plot_scenario(axs[0, 1], f'Peak Shaving (Cap @ {peak_threshold}kW)', load_peak_shaving, generation, time)
plot_scenario(axs[1, 0], 'Load Shifting (Peak to Evening)', load_shifted_evening, generation, time)
plot_scenario(axs[1, 1], f'Efficiency Improvement ({int((1-efficiency_factor)*100)}%)', load_efficiency, generation, time)

# Add shared labels
for ax in axs[:, 0]:
    ax.set_ylabel('Power (kW)')
for ax in axs[1, :]:
    ax.set_xlabel('Time (hours)')

# Create a single legend for the figure
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.95), ncol=4, fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.93]) # Adjust layout to prevent overlap and make space for suptitle/legend
plt.show()


Frequenscy VS imbalanced grid

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Time axis (hours)
time = np.linspace(0, 24, 100)

# Simulate demand (typical daily profile)
demand = 50 + 30 * np.sin(2 * np.pi * time/24 - np.pi/2)

# Simulate supply scenarios
supply_balanced = demand  # Perfect match
supply_under = demand - 10  # Constant under-supply
supply_over = demand + 8  # Constant over-supply
supply_volatile = demand + 10 * np.random.randn(len(time))  # Unstable supply

# Calculate imbalance
imbalance_under = supply_under - demand
imbalance_over = supply_over - demand
imbalance_volatile = supply_volatile - demand

# Create figure
plt.figure(figsize=(10, 6))

# Plot demand and supply
plt.subplot(2, 1, 1)
plt.plot(time, demand, label='Demand', linewidth=2)
plt.plot(time, supply_under, '--', label='Under-Supply')
plt.plot(time, supply_over, '--', label='Over-Supply')
plt.plot(time, supply_volatile, '--', label='Volatile Supply')
plt.ylabel('Power (MW)')
plt.title('Grid Supply-Demand Imbalance')
plt.legend()
plt.grid(True)

# Plot frequency deviation (simplified)
plt.subplot(2, 1, 2)
plt.plot(time, 50 + 0.5*imbalance_under, label='Under-Supply')
plt.plot(time, 50 + 0.5*imbalance_over, label='Over-Supply')
plt.plot(time, 50 + 0.5*imbalance_volatile, label='Volatile Supply')
plt.axhline(50, color='black', linestyle='--', label='Nominal (50Hz)')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (hours)')
plt.title('Grid Frequency Deviation')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('grid_imbalance.png', dpi=300)
plt.show()

Storage vs demand side management

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Regenerate Base Data for Consistent Comparison (from DSM plot) ---
np.random.seed(42) # for reproducibility
time = np.linspace(0, 24, 100)
time_step = time[1] - time[0] # Hours per time step

# Realistic demand
base_demand = 80
peak_demand_factor = 45 * np.sin(np.pi * (time - 8) / 12)
demand_noise = np.random.normal(0, 4, time.shape)
demand = base_demand + np.maximum(0, peak_demand_factor) + demand_noise
demand = np.maximum(demand, 20)

# Realistic wind
wind_profile = 55 * (np.sin(2 * np.pi * (time - 6) / 24) + 0.8) / 1.8
wind_noise = np.random.normal(0, 6, time.shape)
wind = np.maximum(0, wind_profile + wind_noise)

# Realistic solar
solar_profile = 70 * np.maximum(0, np.cos(2 * np.pi * (time - 12.5) / 24))**1.8
solar_noise = np.random.normal(0, 5, time.shape)
cloud_factor = np.maximum(0.3, 1 - 0.7 * np.random.rand(time.shape[0])**2)
solar = np.maximum(0, solar_profile * cloud_factor + solar_noise)

# Total renewable generation
generation = wind + solar

# --- Scenario 1: Simulate Battery Storage ---
storage_capacity_kwh = 200 # Example: 200 kWh capacity
storage_max_power_kw = 50   # Example: 50 kW charge/discharge limit
storage_efficiency = 0.9    # Round-trip efficiency
sqrt_efficiency = np.sqrt(storage_efficiency)

soc_kwh = np.zeros_like(time) # State of Charge (kWh)
storage_charge_kw = np.zeros_like(time)
storage_discharge_kw = np.zeros_like(time)
# soc_kwh[0] = storage_capacity_kwh / 2 # Start half full? Or empty? Let's start empty.

for i in range(len(time) - 1):
    mismatch_kw = generation[i] - demand[i] # Surplus (+) or Deficit (-)
    
    if mismatch_kw > 0: # Surplus -> Try to charge
        charge_power_limit = storage_max_power_kw
        soc_space_kwh = storage_capacity_kwh - soc_kwh[i]
        energy_limit_charge_kw = soc_space_kwh / time_step / sqrt_efficiency # Max power based on remaining space & efficiency
        
        actual_charge_kw = min(mismatch_kw, charge_power_limit, energy_limit_charge_kw)
        storage_charge_kw[i] = actual_charge_kw
        soc_kwh[i+1] = soc_kwh[i] + actual_charge_kw * time_step * sqrt_efficiency
    
    else: # Deficit -> Try to discharge
        deficit_kw = -mismatch_kw
        discharge_power_limit = storage_max_power_kw
        energy_limit_discharge_kw = soc_kwh[i] / time_step * sqrt_efficiency # Max power based on available energy & efficiency
        
        actual_discharge_kw = min(deficit_kw, discharge_power_limit, energy_limit_discharge_kw)
        storage_discharge_kw[i] = actual_discharge_kw
        soc_kwh[i+1] = soc_kwh[i] - actual_discharge_kw * time_step / sqrt_efficiency
        
    # Ensure SoC stays within bounds (due to potential float precision issues)
    soc_kwh[i+1] = np.clip(soc_kwh[i+1], 0, storage_capacity_kwh)

# Calculate net demand after storage and remaining mismatch
net_demand_with_storage = demand - storage_discharge_kw + storage_charge_kw
final_mismatch_storage = generation - net_demand_with_storage
final_deficit_storage = np.maximum(0, -final_mismatch_storage)
final_surplus_storage = np.maximum(0, final_mismatch_storage) # Curtailment

# --- Scenario 2: Apply DSM (Load Shifting to Evening) ---
peak_threshold = 110 # Consistent with previous plot
peak_indices = np.where((time >= 12) & (time <= 18) & (demand > peak_threshold))[0]
target_shift_indices = np.where((time >= 18) & (time <= 22))[0]
load_shifted_evening = demand.copy()
energy_to_shift = 0
if len(peak_indices) > 0:
    energy_above_threshold = (demand[peak_indices] - peak_threshold) * time_step
    energy_to_shift = np.sum(energy_above_threshold)
    load_shifted_evening[peak_indices] = peak_threshold
if len(target_shift_indices) > 0 and energy_to_shift > 0:
    target_duration = len(target_shift_indices) * time_step
    power_increase_target = energy_to_shift / target_duration
    load_shifted_evening[target_shift_indices] += power_increase_target

# Calculate mismatch for DSM scenario
final_mismatch_dsm = generation - load_shifted_evening
final_deficit_dsm = np.maximum(0, -final_mismatch_dsm)
final_surplus_dsm = np.maximum(0, final_mismatch_dsm)

# --- Create 2x1 Subplots for Comparison ---
fig, axs = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
fig.suptitle('Comparing Storage and DSM (Load Shifting) for Grid Balancing', fontsize=16)

# --- Plot Storage Scenario ---
ax1 = axs[0]
ax1.plot(time, demand, label='Original Demand', color='black', linestyle=':', alpha=0.6)
ax1.plot(time, generation, label='Renewable Gen', color='green', alpha=0.7)
# Plot net effect of storage (charge as negative load, discharge as positive)
ax1.fill_between(time, 0, storage_charge_kw, label=f'Storage Charging (Max: {storage_max_power_kw}kW)', color='lightblue', alpha=0.7)
ax1.fill_between(time, 0, -storage_discharge_kw, label=f'Storage Discharging (Max: {storage_max_power_kw}kW)', color='orange', alpha=0.7)
# Show remaining deficit/surplus after storage
ax1.fill_between(time, 0, final_deficit_storage, label='Remaining Deficit', color='red', alpha=0.5, hatch='//')
ax1.fill_between(time, 0, -final_surplus_storage, label='Remaining Surplus (Curtailment)', color='lightgreen', alpha=0.5, hatch='xx')
ax1.set_ylabel('Power (kW)')
ax1.set_title(f'Scenario 1: Balancing with Storage ({storage_capacity_kwh}kWh / {storage_max_power_kw}kW)')
ax1.grid(True, linestyle=':', alpha=0.6)
ax1.legend(loc='upper left', fontsize=9)
ax1.axhline(0, color='black', linewidth=0.5)

# Add SoC on secondary axis
ax1b = ax1.twinx()
ax1b.plot(time, soc_kwh, label='State of Charge (kWh)', color='purple', linestyle='--')
ax1b.set_ylabel('State of Charge (kWh)', color='purple')
ax1b.tick_params(axis='y', labelcolor='purple')
ax1b.set_ylim(0, storage_capacity_kwh * 1.05)
ax1b.legend(loc='upper right', fontsize=9)

# --- Plot DSM Scenario ---
ax2 = axs[1]
ax2.plot(time, demand, label='Original Demand', color='gray', linestyle=':', alpha=0.7)
ax2.plot(time, generation, label='Renewable Gen', color='green', alpha=0.7)
ax2.plot(time, load_shifted_evening, label='Shifted Demand', color='black', linestyle='-')
# Fill the deficit/surplus based on shifted demand
ax2.fill_between(time, load_shifted_evening, generation, where=load_shifted_evening > generation, 
                 label='Deficit after DSM', color='red', alpha=0.4)
ax2.fill_between(time, load_shifted_evening, generation, where=generation >= load_shifted_evening, 
                 label='Surplus after DSM', color='lightgreen', alpha=0.4)
ax2.set_ylabel('Power (kW)')
ax2.set_xlabel('Time (hours)')
ax2.set_title('Scenario 2: Balancing with DSM (Load Shifting to Evening)')
ax2.grid(True, linestyle=':', alpha=0.6)
ax2.legend(loc='upper left', fontsize=9)
ax2.set_ylim(bottom=0) # Ensure y-axis starts at 0

plt.tight_layout(rect=[0, 0, 1, 0.95]) # Adjust layout for suptitle
# plt.savefig('storage_vs_dsm_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
